In [ ]:
%load_ext autoreload
%autoreload 2

import misc
import config
import numpy as np
import blueprint
import tensorflow as tf
import visualization

layers_indexes = config.get_layers_indexes(7)
config = config.Config()


In [ ]:
genome = np.array([[1,1,1,0,1,1,0,0,1,1,1,0,0,1,1,0,1,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0]])
module_genome = misc.module_convert(genome[0], layers_indexes)
module_graph = misc.get_graph(module_genome)


In [ ]:
genome_conv = misc.genome_convert(genome[0], config)
dot = visualization.visualize_genome_main(genome_conv, config)



In [ ]:
print(genome_conv)

In [ ]:
b = blueprint.Blueprint(genome[0], config)


In [ ]:
b.model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
b.model.fit(config.ds_train,
                    epochs=5,
                    use_multiprocessing=False,
                    validation_data=config.ds_test,
                    batch_size=32)

In [ ]:
b.model.summary()

In [ ]:
b.model.evaluate(config.ds_test)

In [ ]:
b.evaluate_model()

In [ ]:
%load_ext tensorboard
import tensorflow as tf


In [ ]:
log_dir = ""
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
b.model.fit(config.ds_train,
          epochs=5, 
          validation_data=(config.ds_test), 
          callbacks=[tensorboard_callback])


In [ ]:
input = tf.keras.Input(shape=config.input_shape, dtype=config.dtype)
last_layer = tf.keras.layers.Flatten()(input)
layer = tf.keras.layers.Dense(units=16,activation='relu',dtype=tf.float32)
last_layer = layer(last_layer)


In [ ]:
output = tf.keras.layers.Dense(units=config.out_units, 
                        name='out_dense_layer',
                        activation=config.out_activation,
                        dtype=config.dtype)(last_layer)

In [ ]:
model2 = tf.keras.Model(input, output)
model2.compile(optimizer=config.optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model2.fit(config.ds_train,
                    epochs=5,
                    use_multiprocessing=True,
                    batch_size=config.batch_size)

In [ ]:
model2.summary()

In [ ]:
model3 = tf.keras.models.Sequential([
  # tf.keras.Input(shape=config.input_shape, dtype=config.dtype),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model3.compile(config.optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model3.fit(
    config.ds_train,
    epochs=5,
    validation_data=config.ds_test,
)


In [ ]:
model4 = tf.keras.models.Sequential([
  tf.keras.Input(shape=config.input_shape, dtype=config.dtype),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model4.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model4.fit(
    config.ds_train,
    epochs=5,
    validation_data=config.ds_test,
)